<a href="https://colab.research.google.com/github/Data-Analytics-with-Python/individual-assignment-iii-kotuzajl/blob/main/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTANT: Before you start, enter your name and student number below.

**Full Name**: Lukas Kotuza-Janisch

**Student Number**: 400238647

# Predictive Analytics for Nata Supermarket

Welcome to Part III of our case assignment. In this part, we will continue working with the same dataset of **Nata Supermarket**.  Our focus here will be on performing predictive modeling tasks.

Throughout this assignment, please ensure that your results are reproducible by setting the **random_state to 20**



# Loading and preparing the data

To begin with, load the data as a `pandas` data frame. Recall that you there are missing values in the data. **Make sure to address** the following issues from part I of the assignment before starting your analysis:

* Remove the missing values
* Remove any column of constant values
* Convert the column `Dt_Customer` to number of days the customer has been with the company.

In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('/W33836-XLS-ENG.csv')
df.head()

# Drop rows with missing values
df = df.dropna()

#remove any column of constant values
df = df.loc[:, (df != df.iloc[0]).any()]

#Convert the column Dt_Customer to number of days the customer has been with the company
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='mixed')

# calculate todays date
todays_date = pd.to_datetime('today')

df['Dt_Customer'] = (todays_date - df['Dt_Customer']).dt.days

df

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
0,5524,1957,Graduation,Single,58138.0,0,0,4978,58,635,...,10,4,7,0,0,0,0,0,0,1
1,2174,1954,Graduation,Single,46344.0,1,1,4132,38,11,...,1,2,5,0,0,0,0,0,0,0
2,4141,1965,Graduation,Together,71613.0,0,0,4479,26,426,...,2,10,4,0,0,0,0,0,0,0
3,6182,1984,Graduation,Together,26646.0,1,0,4072,26,11,...,0,4,6,0,0,0,0,0,0,0
4,5324,1981,PhD,Married,58293.0,1,0,4328,94,173,...,3,6,5,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10870,1967,Graduation,Married,61223.0,0,1,4548,46,709,...,3,4,5,0,0,0,0,0,0,0
2236,4001,1946,PhD,Together,64014.0,2,1,4068,56,406,...,2,5,7,0,0,0,1,0,0,0
2237,7270,1981,Graduation,Divorced,56981.0,0,0,4322,91,908,...,3,13,6,0,1,0,0,0,0,0
2238,8235,1956,Master,Together,69245.0,0,1,4323,8,428,...,5,10,3,0,0,0,0,0,0,0


# Question 1 (45 points)

In this question, we will predict a customer's spending amount on each product category over a two year period. Let us assume that when we try to predict a customer's spending on a product category (such as wines), their spending on other products is not observable.

In this question and Question 2, we will focus on **wines**.

(i). **Split the data** into two data frames, X (**features**) and y (**target**).

Then, further **split the data** into **training** and **testing** sets.

In [74]:
from sklearn.model_selection import train_test_split

# Define the target variable y (MntWines)
y = df['MntWines']

# Define the features X (all other columns except MntWines)
X = df.drop(columns=['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'ID', 'Response'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

(ii). **Build a machine learning pipeline** that combines the following steps to predict spending amount on wines:

* Performing one-hot encoding for the categorical features;
* A random forest model for regression.

In the random forest model, **specify** the following hyperparameters:
* Number of trees;
* Maximum depth of any tree
* Minimum number of data points required to split a node;
* Minimum number of data points in any leaf node

In addition, **fit your model** to the training data.

In [75]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Define the categorical and numerical features

cat_cols = ['Education', 'Marital_Status']
num_cols = ['Year_Birth', 'Kidhome', 'Teenhome','Income','Dt_Customer','Recency','NumDealsPurchases','NumWebPurchases','NumCatalogPurchases','NumStorePurchases','NumWebVisitsMonth','AcceptedCmp1','AcceptedCmp2','AcceptedCmp3','AcceptedCmp4','AcceptedCmp5','Complain']

# Define the column transformer

ct = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown='ignore'), cat_cols),
    ],
    remainder="passthrough"  # Pass through numerical columns without scaling
)

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(
    n_estimators = 100,
    max_depth = 10,
    min_samples_split = 5,
    min_samples_leaf = 3,
    random_state = 20
)

from sklearn.pipeline import Pipeline

pipe = Pipeline(
    steps=[
        ("preprocess", ct),
        ("model", rf)
    ]
)

pipe

pipe.fit(X_train,y_train)

pipe

/usr/local/lib/python3.12/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning:


The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).




Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Education',
                                                   'Marital_Status'])])),
                ('model',
                 RandomForestRegressor(max_depth=10, min_samples_leaf=3,
                                       min_samples_split=5, random_state=20))])

(iii). Use the model to **predict** the spending amount on wines by a

---

customer with the following features.

| Feature |  Value  |
|---------|---------|
| Age     | 48      |
|Education|Graduation|
|Marital_Status| Married|
|Income|80,000|
|Kidhome|1|
|Teenhome|1|
|Dt-Customer|2016-10-10|
|Recency|43|     
|NumDealsPurchases|2|
|   NumWebPurchases|1|
|NumCatalogPurchases|0|
|NumStorePurchases|15|
|NumWebVisitsMonth|5|
|AcceptedCmp1,2,3,4,5| 0 |
|Complain|0|




In [76]:
from sklearn.metrics import confusion_matrix, classification_report

# Predict on X_test
#y_pred = pipe.predict(X_test)

#cm = confusion_matrix(y_test,y_pred)
#print(cm)

#cr = classification_report(y_test,y_pred)
#print(cr)

#(iii). Use the model to predict the spending amount on wines by a customer with the following features.
new_customer = {'Year_Birth': todays_date.year - 48,
                'Education': 'Graduation',
                'Marital_Status': 'Married',
                'Income': 80000,
                'Kidhome': 1, 'Teenhome': 1,
                'Dt_Customer': pd.to_datetime('2016-10-10'), 'Recency': 43,
                'NumDealsPurchases': 2,
                'NumWebPurchases': 1, 'NumCatalogPurchases': 0, 'NumStorePurchases': 15, 'NumWebVisitsMonth': 5, 'AcceptedCmp3': 0, 'AcceptedCmp4':0, 'AcceptedCmp5':0, 'AcceptedCmp1':0, 'AcceptedCmp2':0, 'Complain': 0
                # Added missing columns based on X_train

               }

#columns = ['Year_Birth', 'Education', 'Marital_Status', 'Kidhome', 'Teenhome','Income','Dt_Customer','Recency','NumDealsPurchases','NumWebPurchases','NumCatalogPurchases','NumStorePurchases','NumWebVisitsMonth','AcceptedCmp1','AcceptedCmp2','AcceptedCmp3','AcceptedCmp4','AcceptedCmp5','Complain']

#Convert the column Dt_Customer to number of days the customer has been with the company from new customer
new_cusomter_df = pd.DataFrame([new_customer])

new_cusomter_df['Dt_Customer'] = (todays_date - new_cusomter_df['Dt_Customer']).dt.days

new_cusomter_df = new_cusomter_df[X.columns]

new_customer_predict = pipe.predict(new_cusomter_df)

#print out result to 2 decimals
print(f"Predicted spending on wines: {new_customer_predict[0]:.2f}")

Predicted spending on wines: 382.67


(iv). **Consider** **two** measures to evaluate the model's performance on the test dataset.

Based on you computational results, how would you describe the model's performance?

In [77]:
from sklearn.metrics import mean_squared_error, r2_score

# Predict on test
y_pred = pipe.predict(X_test)

# Metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R-squared (R^2): {r2:.2f}")

Mean Squared Error (MSE): 22063.36
R-squared (R^2): 0.78


he average squared prediction error is approximately 22,063. Taking the square root gives an RMSE of ≈ 148.5, meaning the model’s predictions are, on average, off by about $148–$149 per customer on wine spending over two years. This is an excellent result given that wine spending in the dataset ranges from 0 to more than $1,400. The model explains 78% of the variance in customers’ wine spending using only demographic, tenure, and behavioral features (no other product spending). This is a strong fit.

(v). **Perform** a 6-fold cross validation with a performance score of your choice.

**Note**: You may need to research on how to specify the performance score for regression models.

In [78]:
from sklearn.model_selection import KFold, cross_val_score

# Define the cross-validation strategy using regular KFold
kf = KFold(n_splits=6, shuffle=True, random_state=20)

# Perform cross-validation and get scores using a regression metric
scores = cross_val_score(pipe, X, y, cv=kf, scoring = 'neg_mean_squared_error', n_jobs=-1)

# For errors (like MSE), it returns the negative value so that maximization means minimizing the error.
# Convert to positive MSE
mse_scores = -scores
print(f"Cross-validation MSE scores: {mse_scores}")
print(f"Mean MSE: {mse_scores.mean():.2f}")
print(f"Standard deviation of MSE: {mse_scores.std():.2f}")

Cross-validation MSE scores: [20205.30566075 21445.10616208 27122.1292142  19505.17839486
 23331.04459355 25045.52881263]
Mean MSE: 22775.72
Standard deviation of MSE: 2690.28


(vi). **Perform** hyperparameter tuning using `GridSearchCV` for the following hyperparameters:

* Number of trees: 50, 100
* Maximum depth of any tree: 5, 10, 15
* Minimum number of data points required to split a node: 3, 6
* Minimum number of data points in any leaf node: 2,4,8


Based on your computational result, **show**:
* the best hyperparameter comination
* the corresponding performance score

In addition, **retrieve** the best model (the one corresponding to the best performance score).

In [79]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for GridSearchCV
# The keys should be in the format 'model__parameter_name' since it's within a pipeline
param_grid = {
    'model__n_estimators': [50, 100], # Fewer estimators for faster initial search
    #'model__criterion': ['gini','entropy'],
    'model__max_depth': [5, 10, 15], # None means no limit
    'model__min_samples_leaf': [2, 4, 8],
    'model__min_samples_split': [3, 6]
}

# Initialize GridSearchCV
# We will use the previously defined 'pipe' (preprocessing + model)


grid_search = GridSearchCV(
    estimator=pipe,
    param_grid = param_grid,
    cv = KFold(n_splits=6, shuffle=True, random_state=20),
    scoring='neg_mean_squared_error', # Changed scoring to a regression metric
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

best_score = grid_search.best_score_

best_parameters = grid_search.best_params_

best_model = grid_search.best_estimator_
#pd.DataFrame(grid_search.cv_results_)

#print best parameters, score and model
print(f"Best parameters: {best_parameters}")
print(f"Best score: {best_score}")
print(f"Best model: {best_model}")

Fitting 6 folds for each of 36 candidates, totalling 216 fits
Best parameters: {'model__max_depth': 15, 'model__min_samples_leaf': 2, 'model__min_samples_split': 3, 'model__n_estimators': 100}
Best score: -23099.307575758372
Best model: Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Education',
                                                   'Marital_Status'])])),
                ('model',
                 RandomForestRegressor(max_depth=15, min_samples_leaf=2,
                                       min_samples_split=3, random_state=20))])


# Question 2 (24 points)

In this question, we will compare the performance of the best model found through `GridSearchCV` in Question 1 with the performance of the linear regression model.

(i). **Construct** a linear regression model using all relevant features and fit it to the training data.

Further, **evaluate** the model's performance on the test data and compare it with the best random forest model found in Question 1, with respect to the two performance considered in Question 1.

**Note**: You may use the same training and testing datasets as in Question 1.

In [80]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Same preprocessor as RF
lr = LinearRegression()

lr_pipe = Pipeline(
    steps=[
        ("preprocess", ct),
        ("model", lr)
    ]
)

lr_pipe.fit(X_train, y_train)

# Predict and evaluate LR
y_pred_lr = lr_pipe.predict(X_test)
mse_lr = mean_squared_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

# metrics (from Q1 iv)
y_pred_rf = best_model.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("Linear Regression:")
print(f"MSE: {mse_lr:.2f}, R^2: {r2_lr:.2f}")

print("Random Forest (Best):")
print(f"MSE: {mse_rf:.2f}, R^2: {r2_rf:.2f}")

Linear Regression:
MSE: 32039.41, R^2: 0.68
Random Forest (Best):
MSE: 21568.54, R^2: 0.78


The Linear Regression model explains 68% of the variance in customers’ two-year wine spending and has an average squared error of 32,039 (RMSE ≈ sqrt(32,039) ~= 179).
The best Random Forest model (obtained via GridSearchCV in Question 1) clearly outperforms it on both metrics.

(ii). Let's further compare the distribution of prediction errors by the two models in the following steps.

**Step 1**. For both the linear regression and the (best) random forest model, compute the absolute residual residual for each record in the test dataset.
  * Note that the absolute residual is distance between the predicted value and actual value, i.e., $|y_{pred}-y_{test}|$.

So we end up with two sets of absolute residuals (one by the linear regression model and the other by the random forest model).

**Step 2**. For each pair of absolute residuals for the same test data point, we can define a point in a scatterplot. Genereate such a scatterplot using `plotly.express` (LR residuals vs. RF residuals).  

**Step 3**. Add a 45 degree reference line to the plot. This can be done using the following codes. (You may need to change `min_val` and `max_val` for better visualization).

```
min_val = 0
max_val = 10
fig.add_shape(
    type="line",
    x0=min_val, y0=min_val,
    x1=max_val, y1=max_val,
    line=dict(color="red", width=2, dash="dash"),
)
```

**Implement the above steps**.

Note that the above steps essentially creates a [Q-Q plot](https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot). How would you interpret the plot (for comparing the two predictive models).

In [81]:
# Absolute residuals
abs_res_lr = np.abs(y_test - y_pred_lr)
abs_res_rf = np.abs(y_test - y_pred_rf)

res_df = pd.DataFrame({'LR Residuals': abs_res_lr, 'RF Residuals': abs_res_rf})

# Scatter plot
fig = px.scatter(res_df, x='LR Residuals', y='RF Residuals',
                 title='Absolute Residuals: LR vs RF')

# 45-degree line
min_val = 0
max_val = max(res_df.max())
fig.add_shape(type="line", x0=min_val, y0=min_val, x1=max_val, y1=max_val,
              line=dict(color="red", width=2, dash="dash"))

fig.show()

This is a Q-Q plot of residuals. If points are mostly below the 45° line, RF has smaller errors (better). If above then LR is better. If even, then Similar performance.

# Question 3. (24 points)

In this question, we will consider a classification problem on customers' spendings on meat products.


(i). For the column that represents customers' spendings on meat products, **calculate** the 33.33% and 66.67% percentiles. (**Hint**: You may use the function `df['MntMeatProducts'].quantile([1/3,2/3])`.)

Based on the two percentiles, **label** each row in the dataset
* If a customer's spending is below the 33.33% percentile, label their spending as "low";

* If a customer's spending is above the 66.67% percentile, label their spending as "high";


* If a customer's spending is between the two percentiles, label their spending as "medium".

In [82]:
meat_percentiles = df['MntMeatProducts'].quantile([1/3, 2/3])

lower_bound = meat_percentiles[1/3]
upper_bound = meat_percentiles[2/3]

print(f"33.33% Percentile for MntMeatProducts is: {lower_bound:.2f}")
print(f"66.67% Percentile for MntMeatProducts is: {upper_bound:.2f}")

def label_meat_spending(spending):
    if spending <= lower_bound:
        return 'low'
    elif spending >= upper_bound:
        return 'high'
    else:
        return 'medium'

df['Meat_Spending_Label'] = df['MntMeatProducts'].apply(label_meat_spending)

print("\nValue counts for Meat_Spending_Label:")
print(df['Meat_Spending_Label'].value_counts())

33.33% Percentile for MntMeatProducts is: 24.00
66.67% Percentile for MntMeatProducts is: 146.33

Value counts for Meat_Spending_Label:
Meat_Spending_Label
low       754
high      739
medium    723
Name: count, dtype: int64


(ii). **Build a K-nearest-neighbors model** to predict the label of a customer's spending on meat products. The model should be part of a machine learning pipeline that preprocesses the data.





In [83]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier

y_meat = df['Meat_Spending_Label']
X_meat = df.drop(columns=['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts',
                          'MntSweetProducts', 'MntGoldProds', 'ID', 'Response', 'Meat_Spending_Label'])

X_train_meat, X_test_meat, y_train_meat, y_test_meat = train_test_split(X_meat, y_meat, test_size=0.2, random_state=20, stratify=y_meat)

categorical_features_meat = X_meat.select_dtypes(include=['object']).columns
numerical_features_meat = X_meat.select_dtypes(include=['int64', 'float64']).columns

preprocessor_meat = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features_meat),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_meat)
    ],
    remainder='passthrough'
)

knn_model = KNeighborsClassifier(n_neighbors=5)

knn_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor_meat),
    ('classifier', knn_model)
])

knn_pipeline.fit(X_train_meat, y_train_meat)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  Index(['Year_Birth', 'Income', 'Kidhome', 'Teenhome', 'Dt_Customer', 'Recency',
       'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases',
       'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp3',
       'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2',
       'Complain'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Education', 'Marital_Status'], dtype='object'))])),
                ('classifier', KNeighborsClassifier())])

(iii). Evaluate the performace of your model in part (ii) on the test data by **generating the classification report**.

Further, **interpret** each number in the classification report based on the current context.

In [84]:
from sklearn.metrics import classification_report

y_pred_meat = knn_pipeline.predict(X_test_meat)

report = classification_report(y_test_meat, y_pred_meat)

print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

        high       0.79      0.84      0.82       148
         low       0.85      0.88      0.87       151
      medium       0.70      0.63      0.66       145

    accuracy                           0.79       444
   macro avg       0.78      0.78      0.78       444
weighted avg       0.78      0.79      0.78       444



The model correctly classifies 79% of all customers into low / medium / high meat spending. This is a very solid result for a real-world marketing segmentation task. The macro average treats all three classes equally. The model performs fairly across the board, there's no extreme bias toward one class. We see the same for weighted average.

# Describe how you used Gen. AI. in this assignment (2 points)

I had initially used RandomForestClassifier instead of rf = RandomForestRegressor, since I copied it over from a lecture example, not realizing I didn't change it. I used AI to debug why I was not getting results I should be expecting, and it found that as the error.

Also from last assignment I used it to determine how to convert dates to days, which I carried over the same formula.

Finally, other than it autofilling in a few spots. I used it for column transformer. To use OneHotEncoder(handle_unknown='ignore') as the best and most correct input.

**Note**: The remaining 5 points will be assigned to readability of the work.